In [ ]:
!pwd
!ls

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.fft import fft
import scipy.signal as signal

In [ ]:
path = "./drive/MyDrive/ELEC301/data"
os.chdir(path)

In [ ]:
# Time Domain Data
data_t = []
label = []
for filename in os.listdir():
    fs, data = wavfile.read(filename)
    if len(data.shape) > 1:
        data = data[:, 0]
    data = data.astype(float)
    data_t.append(data)
    label.append(str(filename[: -7]))


IndexError: ignored

In [ ]:
data_pad = np.zeros((len(L), max(L))) # Time Domain Data
for i in range(0, len(L)):
    data_pad[i, range(0, L[i])] = data_t[i]
print(data_pad)

In [ ]:
# Spectrogram
DFT = []
for i in range(1125):
  seg = []
  for j in range(6):
    f,t,Sxx = signal.spectrogram(D[i][j], fs=48000, nperseg=1024)
    Sxx_cut = Sxx[range(46)]
    Sxx_final = Sxx_cut[:,range(0,46)]
    seg.append(Sxx_final)
  DFT.append(seg)

In [ ]:
print(DFT[0][0].shape)
print(t.shape)
print(f[45])
fig, ax = plt.subplots(1,1)
ax.pcolormesh(t[range(46)],f[range(46)],DFT[10][1],shading='gouraud')

In [ ]:
DFT = np.array(DFT)
DFT.shape

(1125, 6, 46, 46)

In [ ]:
DATA = np.zeros((1125,46,46,6))
for i in range(1125):
  for j in range(46):
    for k in range(46):
      for m in range(6):
        DATA[i,j,k,m] = DFT[i,m,j,k]

In [ ]:
# Put Data Into 2D Matrix
D = []
for i in range(1125):
  seg_data = []
  for j in range(6):
    data = data_pad[i]
    seg = int(len(data)/(6))
    if j < (5):
      seg_data.append(data[j*seg:j*seg+seg])
    else:
      seg_data.append(data[5*seg:-3])
  D.append(seg_data)

In [ ]:
D[0][1].shape

(42175,)

In [ ]:
!pip install python_speech_features

In [ ]:
# MFCC
from python_speech_features import mfcc
MFCC = []
for i in range(len(data_pad)):
    features_mfcc = mfcc(data_pad[i], fs, nfft=1200)
    features_mfcc = features_mfcc[:, 1:]
    features_mfcc = features_mfcc.T
    #mfcc_processed = np.mean(features_mfcc,axis=1)
    #mid.append(features_mfcc)
    MFCC.append(features_mfcc)

In [ ]:
MFCC = np.array(MFCC)
MFCC.shape

(1125, 12, 526)

In [ ]:
print(type(MFCC[0]))
MFCC1= np.zeros((1125,3,3,12))
for i in range(1125):
  for j in range(3):
    for k in range(3):
      MFCC1[i,j,k] = MFCC[i,3*j+k,:]
print(MFCC1[0][1][1])

<class 'numpy.ndarray'>


IndexError: ignored

In [ ]:
# Make Number Label
Y = []
for emo in range(0, len(label)):
    if label[emo] == 'angry':
      Y.append(0)
    if label[emo] == 'calm':
      Y.append(1)
    if label[emo] == 'disgust':
      Y.append(2)
    if label[emo] == 'fearful':
      Y.append(3)
    if label[emo] == 'happy':
      Y.append(4)
    if label[emo] == 'neutral':
      Y.append(5)
    if label[emo] == 'sad':
      Y.append(6)
    if label[emo] == 'surprised':
      Y.append(7)

In [ ]:
DATA[0].shape

(46, 46, 6)

In [ ]:
# Select Training and Test Data
#MFCC = MFCC1
Y = np.array(Y)
random_traning_data = np.random.choice(len(Y), 500, replace=False)
#train_data = DATA[random_traning_data]
train_data = MFCC[random_traning_data]
train_label = Y[random_traning_data]
remaining_data = np.setdiff1d(range(0, len(Y)), random_traning_data)
#test_data = DATA[remaining_data]
test_data = MFCC[remaining_data]
test_label = Y[remaining_data]
len(test_label)+len(train_label)

1125

In [ ]:
train_data

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (2, 2), activation='softmax', input_shape=(12, 526, 1)))
model.add(layers.MaxPooling2D((1, 2)))
model.add(layers.Conv2D(32, (2, 2), activation='relu'))
model.add(layers.MaxPooling2D((1, 2)))
model.add(layers.Conv2D(32, (2, 2), activation='relu'))
model.add(layers.MaxPooling2D((1, 2)))
model.add(layers.Conv2D(32, (2, 2), activation='relu'))
model.add(layers.MaxPooling2D((1, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(8))
model.summary()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
history = model.fit(train_data, train_label, epochs=15, 
                    validation_data=(test_data, test_label))

Epoch 1/15
16/16 [==============================] - 7s 408ms/step - loss: 2.0607 - accuracy: 0.1700 - val_loss: 2.0220 - val_accuracy: 0.1552
Epoch 2/15
16/16 [==============================] - 6s 396ms/step - loss: 1.9380 - accuracy: 0.2560 - val_loss: 1.9636 - val_accuracy: 0.2416
Epoch 3/15
16/16 [==============================] - 7s 414ms/step - loss: 1.7294 - accuracy: 0.3720 - val_loss: 1.8493 - val_accuracy: 0.3168
Epoch 4/15
16/16 [==============================] - 6s 395ms/step - loss: 1.4342 - accuracy: 0.5560 - val_loss: 1.8195 - val_accuracy: 0.3600
Epoch 5/15
16/16 [==============================] - 6s 396ms/step - loss: 1.2445 - accuracy: 0.6060 - val_loss: 1.8260 - val_accuracy: 0.3776
Epoch 6/15
16/16 [==============================] - 8s 536ms/step - loss: 1.0277 - accuracy: 0.6440 - val_loss: 1.5998 - val_accuracy: 0.4416
Epoch 7/15
16/16 [==============================] - 6s 406ms/step - loss: 0.7349 - accuracy: 0.7620 - val_loss: 1.7383 - val_accuracy: 0.4320
Epoch 

KeyboardInterrupt: ignored

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
test_loss, test_acc = model.evaluate(test_data,  test_label, verbose=2)
print(test_acc)

In [ ]:
path = "/content/drive/MyDrive/ELEC301/test"
os.chdir(path)

In [ ]:
# Time Domain Data
data_test = []
label_test = []
NUM = []
L_test = [] # length of each audio
for filename in os.listdir():
    label_test.append(str(filename[: -7]))
    fs, data = wavfile.read(filename)
    if len(data.shape) > 1:
        data = data[:, 0]
    data = data.astype(float)
    data_test.append(data)
    L_test.append(len(data))
    NUM.append(str(filename[6: 9]))
print(L_test)

In [ ]:
L_test

In [ ]:
data_pad_test = np.zeros((len(L_test), max(L))) # Time Domain Data
for i in range(0, len(L_test)):
    data_pad_test[i, range(0, L_test[i])] = data_test[i]
print(data_pad_test.shape)

NameError: ignored

In [ ]:
D_test = []
for i in range(315):
  seg_data = []
  for j in range(6):
    data = data_pad_test[i]
    seg = int(len(data)/(6))
    if j < (5):
      seg_data.append(data[j*seg:j*seg+seg])
    else:
      seg_data.append(data[5*seg:-3])
  D_test.append(seg_data)

In [ ]:
# Spectrogram
DFT_test = []
for i in range(315):
  seg = []
  for j in range(6):
    f,t,Sxx = signal.spectrogram(D_test[i][j], fs=48000, nperseg=1024)
    Sxx_cut = Sxx[range(46)]
    Sxx_final = Sxx_cut[:,range(0,46)]
    seg.append(Sxx_final)
  DFT_test.append(seg)

In [ ]:
Y_test = np.array(Y_test)
DFT_test = np.array(DFT_test)
DATA_test = np.zeros((315,46,46,6))
for i in range(315):
  for j in range(46):
    for k in range(46):
      for m in range(6):
        DATA_test[i,j,k,m] = DFT_test[i,m,j,k]

In [ ]:
DATA_test.shape

In [ ]:
label_test

In [ ]:
predictions = model.predict(DATA_test) # 1125 * 48 *28 *10 for DFT, MFCC 还没做
score = tf.nn.softmax(predictions)
label_predict = []
for i in range(315):
  label_predict.append(np.argmax(score[i]))

label_predict

In [ ]:
class_names = np.array(['angry', 'calm', 'disgust', 'fearful', 'happy', 'neutral','sad', 'surprised'])

In [ ]:
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

This image most likely belongs to happy with a 39.44 percent confidence.


In [ ]:
res = []
for emo in range(0, len(label_predict)):
    if label_predict[emo] == 0:
      res.append('angry')
    if label_predict[emo] == 1:
      res.append('calm')
    if label_predict[emo] == 2:
      res.append('disgust')
    if label_predict[emo] == 3:
      res.append('fearful')
    if label_predict[emo] == 4:
      res.append('happy')
    if label_predict[emo] == 5:
      res.append('neutral')
    if label_predict[emo] == 6:
      res.append('sad')
    if label_predict[emo] == 7:
      res.append('surprised')
print(res)

In [ ]:
res = np.array(res).transpose()
NUM = np.array(NUM).transpose()
sample = np.column_stack([NUM, res])
sort_sample = sample[sample[:, 0].argsort()]
name = []
for item in sort_sample[:,0]:
  item = 'sample'+str(item)
  name.append(item)
name = np.array(name).transpose()
sort_sample[:,0] = name
sort_sample

In [ ]:
import pandas as pd
DF = pd.DataFrame(sort_sample)
DF.to_csv("res.csv")